In [1]:
import requests
import pandas as pd
from selenium import webdriver
import os
import re
import pandas as pd

In [2]:
url = 'https://www.streetcheck.co.uk/postcode/startingwith/e17'

In [3]:
# Iterate through files to find path of driver 

is_folder = False

start = os.path.dirname(os.path.abspath(os.curdir))
chrome_driver_path= None
while not is_folder:
    if 'Development' in os.listdir(start):
        filepath=start
        is_folder=True
        chrome_driver_path = os.path.join(start,'Development', 'chromedriver')
        if 'chromedriver' in os.listdir(os.path.dirname(chrome_driver_path)):
            print('chrome driver found')        
    else:
        start=os.path.dirname(start)

chrome driver found


In [4]:
driver = webdriver.Chrome(executable_path=chrome_driver_path)

In [5]:
driver.get(url)

In [6]:
address_block = driver.find_element_by_id("main_inner")

In [8]:
# case sensitive, i.e. postcode must be upper case and must also contain a space
postcode_regex = '([A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2})'

In [9]:
# TODO: Find a way to get all the text that shows "no longer in use"? might be able to 
# eliminate old addresses by using the postcodes.io API

addresses = []

for i in address_block.find_elements_by_tag_name('p'):
    d = {}
#         print(i.text)
    d['paragraph_text'] = i.text
    for j in i.find_elements_by_tag_name('a'):
        d['link_text'] = j.text
        d['link'] = j.get_attribute('href')
        addresses.append(d)
#             print(j.text, j.get_attribute('href'))

In [12]:
# TODO: Flag postcodes that are no longer in use

# remove non address row 
addresses_cleaned = [a for a in addresses if re.search(postcode_regex, a['paragraph_text'])]

for a in addresses_cleaned:
    if re.search('no longer in use', a['paragraph_text'].lower()):
        a['in_use'] = 0
    else:
        a['in_use'] = 1

In [13]:
# TODO: Convert to list of dicts to data frame
df = pd.DataFrame(addresses_cleaned)

In [14]:
df[df['in_use']==1].head()

,in_use,link,link_text,paragraph_text
30,1,https://www.streetcheck.co.uk/postcode/e170bp,"Waltham Forest, London, E17 0BP","Waltham Forest, London, E17 0BP"
103,1,https://www.streetcheck.co.uk/postcode/e170gf,"Waltham Forest, London, E17 0GF","Waltham Forest, London, E17 0GF"
115,1,https://www.streetcheck.co.uk/postcode/e170gx,"Waltham Forest, London, E17 0GX","Waltham Forest, London, E17 0GX"
129,1,https://www.streetcheck.co.uk/postcode/e170hq,"Waltham Forest, London, E17 0HQ","Waltham Forest, London, E17 0HQ"
178,1,https://www.streetcheck.co.uk/postcode/e170nb,"Waltham Forest, London, E17 0NB","Waltham Forest, London, E17 0NB"


In [15]:
df.describe()

,in_use
count,2672.000000
mean,0.555015
std,0.497057
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [16]:
df.to_csv('e17_addresses.csv', index = False)

In [17]:
# driver.close() # this will close a specific tab
driver.quit() # this will close the entire browser

In [ ]:
# TODO: geocode all postcodes